# 通过 BscScan API 获取钱包交易记录

In [85]:
import requests
import time

In [9]:
API_KEY = "SWU95Q2N98X2BYXR88B9KZKKDQ41YE3WWY"
ADDRESS = "0xd3a22590f8243f8e83ac230d1842c9af0404c4a1"
URL = f"https://api.bscscan.com/api?module=account&action=txlist&address={ADDRESS}&startblock=0&endblock=99999999&sort=desc&apikey={API_KEY}"

In [6]:
def fetch_transactions():
    retries = 3
    for i in range(retries):
        try:
            response = requests.get(URL, timeout=10)
            response.raise_for_status()  # 抛出 HTTP 错误
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"请求失败，第 {i+1} 次尝试，错误: {e}")
            time.sleep(2)  # 等待2秒重试
    return None

In [54]:
data = fetch_transactions()
if data:
    print("成功获取交易数据:\n")
    for transaction in data['result'][:5]:
        print(transaction)
        print()
else:
    print("获取数据失败，请检查 API Key 或网络")

成功获取交易数据:

{'blockNumber': '46242945', 'timeStamp': '1738310162', 'hash': '0x0aec27e09a3c5a3474b9f58b2bad61e93c2b46566c092fbc1a27f50dd5495628', 'nonce': '175', 'blockHash': '0xe440493579edeaf2ebb2f69bf2823b8889723cf69fd7929a5a2819e8d890ce4e', 'transactionIndex': '91', 'from': '0xd3a22590f8243f8e83ac230d1842c9af0404c4a1', 'to': '0x0d8ce2a99bb6e3b7db580ed848240e4a0f9ae153', 'value': '0', 'gas': '71946', 'gasPrice': '1000000000', 'isError': '0', 'txreceipt_status': '1', 'input': '0xa9059cbb00000000000000000000000008439901c2bb071cd0812ed329675c965743408300000000000000000000000000000000000000000000a968163f0a57b4000000', 'contractAddress': '', 'cumulativeGasUsed': '6176571', 'gasUsed': '41783', 'confirmations': '24688', 'methodId': '0xa9059cbb', 'functionName': 'transfer(address _to, uint256 _value)'}

{'blockNumber': '46160241', 'timeStamp': '1738062029', 'hash': '0x9a232c8fe83c0cf8e0dd042a36bc0c3af88b29b04098c5c0623bc6239ec4aa54', 'nonce': '1947', 'blockHash': '0x97924b0b6c705ad7a64bdb3782

In [55]:
if data and data.get("status") == "1":
    # Extract transaction details
    transactions = data.get("result", [])
    print(f"Fetched {len(transactions)} transactions.")

Fetched 241 transactions.


# 将获取的数据放入csv，并导入人工交易记录

In [61]:
import csv

def save_transactions_to_csv(transactions, filename="transactions.csv"):
    fieldnames = ["hash", "from", "to", "value", "timestamp"]
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for txn in transactions:
            writer.writerow({
                "hash": txn["hash"],
                "from": txn["from"],
                "to": txn["to"],
                "value": txn["value"],
                "timestamp": txn["timeStamp"]
            })


In [57]:
save_transactions_to_csv(data['result'][:20])
print('cvs filed saved, check directory')

cvs filed saved, check directory


In [80]:
import pandas as pd

# Load the internal transaction data
internal_data = pd.read_csv("internal_data.csv")
print('internal_data is loaded')

internal_data is loaded


In [81]:
print('the "reported_value" is our manual input')
print(internal_data[:3])


the "reported_value" is our manual input
                                                hash  \
0  0x0aec27e09a3c5a3474b9f58b2bad61e93c2b46566c09...   
1  0x9a232c8fe83c0cf8e0dd042a36bc0c3af88b29b04098...   
2  0x3e025f30119ec86bdddd83777f33bf7fdaacc48e4950...   

                                         from  \
0  0xd3a22590f8243f8e83ac230d1842c9af0404c4a1   
1  0x08439901c2bb071cd0812ed329675c9657434083   
2  0xd3a22590f8243f8e83ac230d1842c9af0404c4a1   

                                           to   timestamp trader_name  \
0  0x0d8ce2a99bb6e3b7db580ed848240e4a0f9ae153  1738310162     TraderA   
1  0xd3a22590f8243f8e83ac230d1842c9af0404c4a1  1738062029     TraderB   
2  0x55d398326f99059ff775485246999027b3197955  1737961174     TraderC   

           reported_value  
0                       0  
1  4846690000000000000000  
2                       0  


# 通过pandas merge 以及 valuematch 来检查交易额度是否一致

In [82]:
blockchain_data = pd.DataFrame(data['result'])

merged_data = pd.merge(blockchain_data, internal_data, on='hash', how='inner')
print(merged_data[:3])

  blockNumber   timeStamp                                               hash  \
0    46242945  1738310162  0x0aec27e09a3c5a3474b9f58b2bad61e93c2b46566c09...   
1    46160241  1738062029  0x9a232c8fe83c0cf8e0dd042a36bc0c3af88b29b04098...   
2    46126623  1737961174  0x3e025f30119ec86bdddd83777f33bf7fdaacc48e4950...   

  nonce                                          blockHash transactionIndex  \
0   175  0xe440493579edeaf2ebb2f69bf2823b8889723cf69fd7...               91   
1  1947  0x97924b0b6c705ad7a64bdb37829014458251fdb683ee...               17   
2   174  0x53724939ed9e3423716f233a9d0ffb98a84c571b7752...               42   

                                       from_x  \
0  0xd3a22590f8243f8e83ac230d1842c9af0404c4a1   
1  0x08439901c2bb071cd0812ed329675c9657434083   
2  0xd3a22590f8243f8e83ac230d1842c9af0404c4a1   

                                         to_x                   value     gas  \
0  0x0d8ce2a99bb6e3b7db580ed848240e4a0f9ae153                       0   71946   
1  

# 成功发现一笔交易额度差异

In [84]:
# Compare the values (note that values might need to be in the same unit)
merged_data['value_match'] = merged_data['value'] == merged_data['reported_value']

# Print the rows where there is a discrepancy
discrepancies = merged_data[merged_data['value_match'] == False]
print(discrepancies)

  blockNumber   timeStamp                                               hash  \
7    45921421  1737345552  0x738d3c05d4afb172e3a1d50a8cee8c00368a1e13a0ed...   

  nonce                                          blockHash transactionIndex  \
7     2  0xcabdc88ccc528547950914d04cb98deb840e4675bcb7...               51   

                                       from_x  \
7  0x084384853199d910d35a0730208af2e5d4234083   

                                         to_x          value    gas  ...  \
7  0xd3a22590f8243f8e83ac230d1842c9af0404c4a1  4000000000000  21000  ...   

  gasUsed confirmations methodId functionName  \
7   21000        346212       0x                

                                       from_y  \
7  0x084384853199d910d35a0730208af2e5d4234083   

                                         to_y   timestamp trader_name  \
7  0xd3a22590f8243f8e83ac230d1842c9af0404c4a1  1737345552     TraderH   

  reported_value value_match  
7  3999999999999       False  

[1 rows x 26 columns

# 最后，如有需要可以update交易数据

In [ ]:
'''
# If the discrepancy is found and corrected

internal_data.loc[internal_data['hash'] == discrepancies['hash'].iloc[0], 'value'] = correct_value

# rerun audit
discrepancies = merged_data[merged_data['value_match'] == False]
print(discrepancies)
'''